Try timestamp input with time HH:MM:SS

! timedelta(days=1) = timedelta(hours=24) -> doesn't change at midnight/partial days

Would do with more time:
Would fix input to find date even if input doesn't match format

In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
# Read in data
# products = pd.read_csv("../Resources/products.csv")
# accounts = pd.read_csv("../Resources/accounts.csv")
ast = pd.read_csv("../Resources/account_state_transitions.csv")
# gpa = pd.read_csv("../Resources/group_product_associations.csv")
# groups = pd.read_csv("../Resources/groups.csv")
# payments = pd.read_csv("../Resources/payments.csv")

In [3]:
ast.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


In [4]:
all_unique_accounts = ast.account_id.unique() # len 772-44=728

## 2) Devise a method of generating a table of all Accounts with their State as of any arbitrary timestamp in the past and the total time they had been in that state as of the timestamp.

### Limit Dataframe by Indicated Timestamp

In [5]:
# for all ACCOUNT_ID, return TO_STATE where STARTED_WHEN = TIMESTAMP OR nearest_preceding_date
# if ACCOUNT_ID doesn't have corresponding TIMESTAMP select earliest preceding date with record (preceding_date)
# select TO_STATE of PRECEDING_DATE and calculate time difference: TIMESTAMP-PRECEDING_DATE

In [6]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-03-01


In [7]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

In [8]:
# Change data type from string to datetime object
ast['started_when'] = pd.to_datetime(ast['started_when'])

In [9]:
# Create limited dataframe based on indicated timestamp 
df = ast.loc[ast['started_when'] <= (timestamp_dt + timedelta(hours=23,minutes=59,seconds=59))]
df.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


In [10]:
# Select max date for each unique account_id
df_grouped = df.groupby('account_id',as_index=False)['started_when'].max()
df_grouped.head()

,account_id,started_when
0,1,2020-01-17 13:01:20.571392
1,2,2020-02-24 02:00:21.571392
2,3,2020-02-21 11:40:47.887181
3,4,2020-02-25 04:28:59.571392
4,5,2020-02-29 20:42:56.571392


In [ ]:
df_grouped.shape
## Need to grab STATE

In [20]:
grouped_accounts = df_grouped['account_id']
grouped_dates = df_grouped['started_when']

In [32]:
# for i in range(0,len(grouped_accounts)):
#     curr_account = grouped_accounts[i]
#     curr_date = grouped_dates[i]


for i in range(0,len(df['account_id'])):
    try:
        print(df['account_id'][i])
    except KeyError:
        continue
        

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
1

180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
185
185
185
185
185
185
185
185
185
185
185
185
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
186
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187
187


373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
373
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
374
376
377
379
380
381
383
384
385
385
386
387
388
389
390
391
393
396
399
401
403
404
405
408
409
410
410
413
414
420
421
423
424
429
430
430
431
431
432
433
435
435
439
445
445
446
446
447
447
449
450
451
452
453
454
455
456
456
458
460
461
462
463
465
467
469
471
472
473
474
477
478
479
480
482
484
485
486
487
489
491
495
496
497
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
499
500
500
500
500
500
500
500
500
500
500
500
500
500
500
504
504
504
504
504
504
504
504
504
504
504
504
504
504
504
504
504
504
504
504
507
507
507
507
507
507
507
507
507
507
507
507
507
507
507
507
507
507
507
508
508
508
508
508
508
509
509
509
509
509
509
509
509
510
510
510
510
510
510
510
510
510
510
510


622
622
622
622
622
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
623
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
624
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
625
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
626
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
627
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
628
629
629
629
629
629
629


362
362
362
362
362
362
362
362
362
362
362
362
362
362
362
362
362
362
362
362
362
362
721
565
50
50
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
591
591
591
250
250
250
250
312
299
299
299
299
299
299
299
299
299
299
299
299
299
299
633
633
633
633
633
633
633
633
633
633
633
633
633
633
633
633
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
73
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
277
268
268
268
268
425
253
253
253
253
253
253
253
253
253
253
253
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
672
751
751
751
751
751
751
751
751
505
505
505
505
505
505
505
505
505
505
505
505
505
505
505
505
416
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
166
32
32
32
32
32
32
32
32
32
32
32
32
32

### Current State of Each Account by Indicated Timestamp

In [ ]:
# Create dataframe to show current state of each account based on indicated timestamp
df_limited2 = df_limited[['started_when','account_id','to_state']]

# Sort by account_id
df_limited2 = df_limited2.sort_values('account_id')

# Reset index
df_limited2 = df_limited2.reset_index(drop=True)
df_limited2.head()

In [ ]:
# Show in dataframe, any accounts occuring more than once during indicated timestamp

account_ids = []
df_repeat = pd.DataFrame(columns=['started_when', 'account_id', 'to_state'])

for i in range(0,len(df_limited2['account_id'])):
    account_id = df_limited2['account_id'][i]
    if account_id in account_ids:
        df_temp = df_limited2.loc[df_limited2['account_id'] == account_id]
        df_repeat = df_repeat.append(df_temp)
    else:
        account_ids.append(account_id)

df_repeat

### Total Time in State as of Indicated Timestamp

In [ ]:
# if total time = start to timestamp:
# for each account id, select state = current state for all records <= timestamp
